# OnePassStep Example Notebook
***

Author: Varun Bajaj<br>
email: vbajaj@stsci.edu<br>
November 2023

This notebook presents a simple example workflow for performing One Pass Style PSF fitting for JWST data, and making a master catalog.  In essence, the process can be broken down into three steps:
 - First, run PSF photometry on each image individually
 - Then, align the images using the resulting catalogs as inputs to the `TweakRegStep`
 - Lastly, use the newly aligned images/catalogs to match stars across the all of the catalogs, and combine the results (calculate means and stds) into a master catalog

This master catalog can then be used as an astrometric reference to align other data, or serve as a high precision photometric catalog.
In general, fluxes an positions for brighter stars in the master catalog are measured to standard deviations (__not__ errors, which are a factor of `sqrt(n)` lower) of approximately 0.01 mag, or 0.015 pixels, 

While this case shows NIRCam shortwave data, doing the same for longwave and MIRI Image data is straightforward, and only requires a few small changes.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> To keep the scope of this notebook small, we do not intend this as a extensive guide to image alignment, but just as a basic intro to the package.  If desired, more detailed tweakreg workflows can be detailed as well.
</div>

In [ ]:
import glob
import matplotlib.pyplot as plt
import os
import shutil

from astroquery.mast import Observations
from astropy.io import fits
from astropy.table import Table, vstack
from jwst.datamodels import ImageModel, ModelContainer
from jwst.pipeline import  calwebb_image3
from matplotlib.colors import LogNorm

import one_pass_fitting
from one_pass_fitting import make_jwst_tweakreg_catfile, merge_catalogs, create_image_handlers
from one_pass_fitting.data_handlers import  NIRCamHandler

In [ ]:
os.environ['WEBBPSF_PATH'] = '/Users/vbajaj/Downloads/webbpsf-data'
import webbpsf

## Basic setup

In [ ]:
# Change as desired
home_dir = os.path.expanduser('~')
data_dir = f'{home_dir}/Documents/OnePassPhot_example_data/JWST'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

os.chdir(data_dir)

In [ ]:
obs = Observations.query_criteria(obs_id='jw01476-o001_t001_nircam_clear-f150w')
plist = Observations.get_product_list(obs)
fplist = Observations.filter_products(plist, productSubGroupDescription='CAL')

# To make the example faster, lets just use the NRC A1 data
# You can replace this data as desired
fplist = vstack([row for row in fplist if row['obs_id'].endswith('nrca1')])
dl_table = Observations.download_products(fplist)

In [ ]:
for im in sorted(glob.glob('./mastDownload/JWST/*/*cal.fits')):
    shutil.move(im, '.')

In [ ]:
cal_ims = sorted(glob.glob('*cal.fits'))

In [ ]:
cal_ims

Let's create the PSF Model using Webb PSF.  Note that the `npsf` argument is set to 9 for speed here, but it's probably better to set it higher (must be a square, so 25 is recommended for real use).

In [ ]:
nrc = webbpsf.NIRCam()

In [ ]:
def make_nircam_psf(detector: str, filt: str, pupil: str, npsf: int = 9):
    """Get GriddedPSFModel for a given combination of nircam detector/optical element"""
    if detector.lower() in ['along', 'blong']:
        detector = detector.lower().replace('long', '5')
    nrc.detector = 'nrc'+detector.upper()
    
    if pupil.upper() == 'CLEAR':
        nrc.filter = filt.lower()
    else:
        nrc.filter = pupil.lower()
    # 9 is likely too low, probably make it 25
    grid = nrc.psf_grid(num_psfs=npsf, all_detectors=False)
    return grid
    


<div class="alert alert-block alert-warning">
For other data, change this as needed.  If doing all 8 nircam detectors for a filter, make sure you're using the right model for each detector!
</div>

In [ ]:
filt = 'F150W'
det = 'A1'
pupil = 'CLEAR'
psf_model = make_nircam_psf(det, filt, pupil, npsf=9)

Now with the PSF model, we can initialized the `OnePassPhot` Object, but first let's take a look at the documentation:

In [ ]:
one_pass_fitting.OnePassPhot?

To get into more detail, the majority of the parameters here are related to source detection.  Specifically:
  - `hmin` is minimum separation a local maximum in the data has to be from any other local max.  This is like a minimum isolation value of source
  - `fmin` is the minimum flux of the sume of brightest 4 pixels of the local max, it is a rough cut to make sure the local max is bright enough
  - `pmax` is the maximum value of the brightest pixel of the local max.  This often can help make sure you're not fitting saturated sources
  - `sky_in` and `sky_out` are used for calculating a local background around the local max, as the PSF fitting requires the sky to be pre-subtracted

In [ ]:
# These are rough parameters, feel free to change as needed for your data
ophot = one_pass_fitting.OnePassPhot(psf_model, hmin=5, fmin=200., pmax=70000.)

To actually run the PSF fit photometry, you can use the `__call__()` method on `OnePassPhot`, for documentation:

In [ ]:
ophot.__call__?

In [ ]:
tbls = []
for im in cal_ims:
    mod = ImageModel(im)
    print(f'PSF fitting {im}')
    # To use the __call__() method, simply put parenthesis after of object, with whatever arguments inside
    output = ophot(mod.data, mod.meta.wcs, im.replace('.fits', '_sci1_xyrd.ecsv'))
    tbls.append(output)

Now that we have these PSF photometry catalogs written out to disk, we can then use these catalogs to do high precision alignments of the data!

In [ ]:
tbls = [Table.read(t) for t in sorted(glob.glob('*xyrd.ecsv'))]

Let's take a look at one the output catalogs:

In [ ]:
tbls[0]

The column meanings are:
 - `x, y`: x, y position in the output frame (pixels)
 - `m`: "instrumental" magnitude (-2.5log(flux)). This is not meaningful for JWST due to the units being MJy/sr, but is for HST.
 - `q`: quality of fit defined as sum(abs(data-model))/flux
 - `s`: local sky value
 - `cx`: central excess value, which the excess of the flux in the central pixel divided by the fit flux, if negative, the model has more flux in the central pixel than the data
 - `f`: Fit flux, in instrumental units
 - `RA, Dec`: Right ascension, declination of the `(x,y)` coordinate

## Aligning images using the PSF fit catalogs

To use the catalogs with the TweakReg step, we must first create a catfile, which lists the image and it's corresponding catalog.  The catalog must be written to disk for this to work.  Let's look at the function docs:

In [ ]:
make_jwst_tweakreg_catfile?

In [ ]:
# This variable will store the name of the catfile which will be passed to the TweakRegStep
catfile = make_jwst_tweakreg_catfile(cal_ims, tbls)

In [ ]:
!cat tweakreg_catfile.txt

Let's actually align the data using the TweakRegStep.  We need to tell the step to use our custom catalogs rather than to do its own detection.  We can really crank down on the tolerance and sigma clipping parameters due to having good PSF fit postions.

Specifically, take a look at the fit MAE (median absolute error) values here- those are in arcsec, and often times are below 1mas!

In [ ]:
im3_pipe = calwebb_image3.Image3Pipeline()

im3_pipe.tweakreg.use_custom_catalogs = True
im3_pipe.tweakreg.catfile = catfile
im3_pipe.tweakreg.fitgeometry = 'general'
im3_pipe.tweakreg.sigma = .3
im3_pipe.tweakreg.tolerance = .08 # Can really crank down on the tolerance because our positions are so precise 

# If the next lines are commented out, no absolute alignment will be performed.
# im3_pipe.tweakreg.abs_refcat = 'GAIADR3'  # Can optionally do post alignment (after aligning to each other) to Gaia.
# im3_pipe.tweakreg.abs_fitgeometry = 'general'
# im3_pipe.tweakreg.abs_tolerance = 0.08
# im3_pipe.tweakreg.abs_sigma = .5
# im3_pipe.tweakreg.abs_nclip = 4
aligned_mc = im3_pipe.tweakreg.run(cal_ims)

## Making a Final Catalog

Now that we have the images aligned using our catalogs, we can now combine the catalogs into one master catalog.  To do that, we first need to create some convenience objects, which we will refer to as image handlers (in this case `NIRCamHandler` objects)

In [ ]:
im_handlers = create_image_handlers([*aligned_mc], tbls)

Now that we have the image handler objects created, we can do some catalog corrections:

In [ ]:
im_handlers[0].correct_catalog?

If you used WebbPSF Models, you can set all the flags to `False`.  You still need to run the method however, as it converts the fluxes to more relevant units.

In [ ]:
for ih in im_handlers:
    ih.correct_catalog(aperture=False, pixel_area=False, encircled=False)

Now that the catalogs are corrected, you can pass these objects into one last method to combine all their catalogs!

In [ ]:
merge_catalogs?

In [ ]:
res = merge_catalogs(im_handlers, match_size=0.65)

Let's take a look at the output merged catalog.  For most of the columns there exists both a `mean` and `std` column.  They represent the average and standard deviation of the matched measurements for each star.

In [ ]:
res.colnames

For the columns that appeared in the tables from running `OnePassPhot`, the columns in the merged catalog are the means/stds of the matched values for each star across all of the input catalogs.  For certain quantities, like `s` and `cx`, the mean and standard deviations may vary, as the PSF and background may often vary across images.

In addition, it adds extra columns `ST_mean` and `AB_mean` (and their std values) which are the magnitudes of the stars in the STMag and ABMag systems, respectively (Vega is coming soon).

Lastly, the `n` and `n_expected` columns count how many times a given source was found across all of the catalogs, while `n_expected` shows the number of images that covered that source's `(RA,Dec)`, which helps give context to `n`.

***

### Cutting the catalog, Exploring results

Even if a star is only found once across all of the catalogs, it is still present in the final catalog.  You may want to remove these, as they have meaningless standard deviations.  You may want to cut out sources detected fewr than a certain number of times, which can be done with standard astropy table logic:

In [ ]:
res_cut = res[res['n']>=2]
# Another option, makes a cut to make sure a star is detected at least twice, and in at least half the images that covered it
# res_cut = res[(res['n']/res['n_expected']>=0.5)&(res['n']>=2)]

Doing some quick analysis, we can see the fit sources are measured fairly well over a range of ~3 magnitudes, before the SNR of the source gets too low to well constrain the flux.

In [ ]:
plt.scatter(res_cut['AB_mean'], res_cut['AB_std'], s=5, alpha=.5)
plt.ylim(0, .1)
plt.xlabel('ABMag [mag]')
plt.ylabel('ABMag std [mag]')
plt.grid(ls=':', alpha=.4)

Similarly the positions are measured down to ~0.01-0.015 pixels for the brighter stars, which is much more precise than typical star finding algorithms

In [ ]:
fig, axs = plt.subplots(2, 1)

axs[0].scatter(res_cut['AB_mean'], res_cut['x_std'], s=5, alpha=.35)
axs[0].set_ylim(0, .05)
axs[0].set_xlabel('ABMag [mag]')
axs[0].set_ylabel('x_std [pixels]')
axs[0].grid(ls=':', alpha=.4)

axs[1].scatter(res_cut['AB_mean'], res_cut['y_std'], s=5, alpha=.35)
axs[1].set_ylim(0, .05)
axs[1].set_xlabel('ABMag [mag]')
axs[1].set_ylabel('y_std [pixels]')
axs[1].grid(ls=':', alpha=.4)

plt.tight_layout()

Let's also take a look at the map of the RA/Decs for the sources and how often the sources were detected

In [ ]:
plt.scatter(res_cut['RA_mean'], res_cut['Dec_mean'], s=8, alpha=.5, c=res_cut['n'])
plt.xlabel('RA')
plt.xlim(plt.xlim()[::-1])
plt.ylabel('Dec')
cb = plt.colorbar()
cb.set_label('number of detections')
plt.grid(ls=':', alpha=.4)

In [ ]:
plt.scatter(res_cut['RA_mean'], res_cut['Dec_mean'], s=8, alpha=.5, c=res_cut['n_expected'])
plt.xlabel('RA')
plt.xlim(plt.xlim()[::-1])
plt.ylabel('Dec')
cb = plt.colorbar()
cb.set_label('number of images covered')
plt.grid(ls=':', alpha=.4)

***
### Saving as a master catalog

Lastly, we can save the merged catalog as a master catalog which can then be used to align other images!  We simply need to have an ecsv file containing RA/Dec columns.  We'll keep a few other columns in there, just so we can have some context for using it later. 

<div class="alert alert-block alert-warning">
<b>NOTE:</b> For the TweakRegStep, the dec column of the reference catalog has to be `DEC`.
</div>

In [ ]:
master_cat = Table(res_cut['RA_mean', 'Dec_mean', 'q_mean', 'AB_mean', 'RA_std', 'Dec_std', 'AB_std', 'n', 'n_expected'],
                   names = ['RA', 'DEC', 'q_mean', 'AB_mean', 'RA_std', 'Dec_std', 'AB_std', 'n', 'n_expected']) 

To save it, we simply use the `write()` method of the Table.

In [ ]:
master_cat.write(f'{filt}_master_cat.ecsv', overwrite=True)

## Conclusion

We hope this has been a good introduction to PSF photometry use cases.  Contributions and feedback are more than welcome, especially for other features and support for other instruments.  This package is still under heavy development, and so you may run into some bugs.  Do not hesitate to reach out to vbajaj@stsci.edu or open issues/PRs on github!

Also feel free to take a look at the documentation to see if there are other methods under the hood that might be useful for you!

<img style="float: center;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>